In [6]:
import os
import random

from pydub import AudioSegment

In [13]:
def split_audo(audio:str,audio_folder:str,sample_length:int,sample_count:int):
    '''
    Данная функция генерирует из одного большого аудио файла много файлов поменьше, 
    из которых будет составляться датасет. Все сгенерированные аудио попадают в отдельную директорию, 
    имеющую название исходного файла. 
    
    ==Input==
    audio - путь к исходному файлу
    audio_folder - директория в котороую будет осуществляться генерация
    sample_length - длина генерируемого примера в секундах
    sample_count - количество примеров
    '''
    # проверяем/создаем директорию audio_folder/filename/
    if not os.path.exists(audio_folder):
        os.makedirs(audio_folder)
    sub_folder = os.path.basename(audio).split('.')[0]
    sample_folder = os.path.join(audio_folder,sub_folder)
    if not os.path.exists(sample_folder):
        os.makedirs(sample_folder)

    Audio = AudioSegment.from_wav(audio) # считываем аудио
    max = Audio.duration_seconds - sample_length - 1 # получаем длину аудио
    for i in range(0,sample_count): # с
        start = random.randint(0, max)
        end = start + sample_length
        sample = Audio[start*1000:end*1000]
        sample_name = sub_folder+f'_{i}_.wav'
        sample_path=os.path.join(sample_folder,sample_name)
        sample.export(sample_path, format="wav")

In [14]:
split_audo("./segregated-urban8K-sounds/air_conditioner/79089-0-0-138.wav","./segregated-urban8K-sounds/split_test",
          1,2)